## What this code does
In short, it is a reverse meme search, that identifies the source of the meme. It takes an image copypasta, extracts the individual *subimages* and compares it with a database of pictures (the database should be made up of copypastas, which is in TODO)

### TODO

### Clean up the code
There are many repetitive import statements. <br\>
The code is saving the picture as file so that you can load it into model. <br\>
Anything that you cannot explain in this code <br\>
Change VGG16 to Xception (because I can't upgrade both TF and keras for reasons)

#### Feature vector robustness check
To what extent the following transformations affects the feature vector?
- crop (a little, add bounding boxes)
- photoshop - e.g. cropping a face onto a body
- rotate the image (a little, a lot)
- add text (different sizes)
- vandalised - scribbling markers over 
- add noise (Gaussian etc)
- compression changes
- recoloring - grey-scale
- picture effects - e.g. twisted picture meme
- special effects - e.g. shining eyes meme

#### Image separation testing
We need to ensure the individual pictures are separated correctly.
- pictures now don't have borders
- pictures are no longer rectangular
- whether does it identify the source of the cropped face

#### Database management
We need to do preprocessing of the database. Currently the feature vector is only calculated when you start this notebook. 

Moreover, since the database of copypasta will not be single images, we need to process that aspect as well. From the copypastas we need to identify its subimages and then calculate their feature vector. There also needs to be some way to associate the feature vector and the location of the subimages to the image copypasta, together with its metadata - in a manner that is scalable.

### import imagenet model

In [1]:
%run image_database_helper.ipynb

Using TensorFlow backend.


In [2]:
model = init_model()

### making a list of all the files

In [3]:
!rm 'imgs/.DS_Store'

rm: imgs/.DS_Store: No such file or directory


In [4]:
images = findfiles("new/")
print(len(images))

24


### Processing pictures

In [5]:
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2

In [6]:
import csv

fieldnames = ['img_file_name',
              'number_of_subimages',
              'subimage_number',
              'x',
              'y',
              'w',
              'h',
              'feature_vector']

import os
if not os.path.exists('index_subimage.csv'):
    with open('index_subimage.csv', 'w') as csvfile:
        db = csv.DictWriter(csvfile, fieldnames=fieldnames)
        db.writeheader()

In [7]:
import subprocess
import csv


for img_name in images:
    path_image_to_analyse = "./new/"+img_name
    print(img_name)
    
    img = cv2.imread(path_image_to_analyse)
    output_boxes = get_bounding_boxes(img)

    for i, box in enumerate(output_boxes):
        [x,y,w,h] = box
        output_img = np.array(img[y:y+h, x:x+w])
        cv2.imwrite("temp.jpg",output_img)
        feature_vector = calc_feature_vector(model, "temp.jpg")

        dict_to_write = {'img_file_name':img_name,
                      'number_of_subimages':len(output_boxes),
                      'subimage_number':i,
                      'x':x,
                      'y':y,
                      'w':w,
                      'h':h,
                      'feature_vector':feature_vector}

        with open('index_subimage.csv', 'a') as csvfile:
            db = csv.DictWriter(csvfile, fieldnames=fieldnames)
            db.writerow(dict_to_write)
    
    subprocess.run("mv ./new/{} ./database/{}".format(img_name,img_name),shell=True)

our-singapore-malaysian-durian-fake-768x791.jpg
dog-meat-satay-post.jpg
Screen Shot 2018-04-02 at 05.19.52 AM.png
Screen-Shot-2017-10-12-at-5.33.37-PM.png
ge14fakenews.png
pm-lee-gst-meme.jpg
casket-1-768x1366.jpg
17626490_10155249769684175_8841223787064818741_n.jpg
city-harvest-church-headline-changed.jpg
17951666_1758817064340674_3343590308811084675_n-768x1365.jpg
sexdoll-768x603.jpg
pmo-lky-hoax.jpg
Bear-hoax-singapore-768x432.jpg
notice-sammyboy-e1520150158500-768x620.jpg
ntucfairpricewarning.jpg
IMG_7EF6C6D23BCD-1-768x1366.jpg
ntuc27e_2x.jpg
flyer-breast-cancer-yishun-fake.jpg
Punggo_Waterway_Terraces_roof_top_floors_collapse_1-768x801.jpg
cross-with-phone-fine-fake-768x930.jpg
fake copy.jpg
Screen-Shot-2018-02-09-at-10.26.30.png
ASS-Prakash-2-768x787.jpg
nude-2.jpg


In [8]:
!cp ./database/* ./new/